In [1]:
import json
import gzip
import pandas as pd
import numpy as np

In [2]:
# title download from The Movie DB
file_name = 'tv_series_ids_01_02_2021.json.gz'

In [3]:
# use gzip to open json.gz file
# result has multiple records and json.loads() is not able to decode more than one. Need to do it record by record.
# Otherwise JSONDecodeError: Extra data: line 2 column 1

with gzip.open(file_name, 'rt', encoding='utf-8') as zipfile:
    json_data = [json.loads(line) for line in zipfile]

In [4]:
# first entry example - all are dictionaries
json_data[0]

{'id': 1, 'original_name': 'プライド', 'popularity': 2.384}

In [5]:
# convert list of dictionaries to a dataframe
df = pd.DataFrame(json_data)

In [6]:
df

,id,original_name,popularity
0,1,プライド,2.384
1,2,Clerks: The Animated Series,11.962
2,3,The Message,0.745
3,4,The Amazing Mrs Pritchard,1.740
4,5,La Job,6.901
...,...,...,...
101935,115748,Scholastic Storybook Treasures - 20 Holiday Ad...,0.000
101936,115749,Color Rush,0.000
101937,115750,Split Ends,0.000
101938,115752,仮面ライダーゼロワン,0.000


In [7]:
# test find specific TV series
df.loc[df['original_name']=='Game of Thrones']

,id,original_name,popularity
1348,1399,Game of Thrones,297.83


In [8]:
# search for partial text
df[df['original_name'].str.contains('Sopran')]

,id,original_name,popularity
1347,1398,The Sopranos,74.516


In [9]:
# separating the list into English titles and other languages using special characters
df['special'] = df['original_name'].str.contains(r'[^\x00-\x7F]+')

In [10]:
# export to excel
df.to_excel('Tv_list.xlsx')

### Making a list of TV titles to review

In [11]:
# There are over 100k titles in the download. Shorten this list based on English and popular TV shows

In [12]:
# create a list of conditions. If no special character and popularity greater than 1. 
# If special character and popularity greater than 20.
conditions = [
    (df['special'] == False) & (df['popularity'] >= 1),
    (df['special'] == True) & (df['popularity'] >= 20),
    (df['special'] == True) & (df['popularity'] < 20),
    (df['popularity'] < 1)
    ]

In [13]:
# create a list of values to assign to the conditions. keep or not
values = ['yes', 'yes', 'no', 'no']

In [14]:
# create a new column and use np.select to assign values to it, using conditions and values
df['keep'] = np.select(conditions, values)

In [15]:
df

,id,original_name,popularity,special,keep
0,1,プライド,2.384,True,no
1,2,Clerks: The Animated Series,11.962,False,yes
2,3,The Message,0.745,False,no
3,4,The Amazing Mrs Pritchard,1.740,False,yes
4,5,La Job,6.901,False,yes
...,...,...,...,...,...
101935,115748,Scholastic Storybook Treasures - 20 Holiday Ad...,0.000,False,no
101936,115749,Color Rush,0.000,False,no
101937,115750,Split Ends,0.000,False,no
101938,115752,仮面ライダーゼロワン,0.000,True,no


In [16]:
# create a subset of the dataframe with TV titles to keep
df_keep = df[df['keep'] == 'yes']

In [17]:
# sort dataframe by popularity
df_keep = df_keep.sort_values(by=['popularity'], ascending=False)

In [20]:
# reset the index and drop the old index column
df_keep.reset_index(inplace = True)
df_keep.drop(columns = 'index', inplace = True)

In [21]:
df_keep

,level_0,id,original_name,popularity,special,keep
0,0,82856,The Mandalorian,1183.816,False,yes
1,1,44217,Vikings,893.852,False,yes
2,2,75006,The Umbrella Academy,736.408,False,yes
3,3,71712,The Good Doctor,711.145,False,yes
4,4,77169,Cobra Kai,648.353,False,yes
...,...,...,...,...,...,...
23344,23344,64682,Born This Way,1.000,False,yes
23345,23345,67120,Gay Skit Happens,1.000,False,yes
23346,23346,66985,Big Easy Motors,1.000,False,yes
23347,23347,66733,Bad Internet,1.000,False,yes
